# Степенной закон
## Импорты

In [ ]:
import json
from io import StringIO
import pandas as pd
import numpy as np
import seaborn as sns
import graph_tool as gt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
sns.set_style("ticks",{'axes.grid' : True})

## Построение DataFrame из JSON

In [ ]:
def to_df(read_file: str, key: str) -> pd.DataFrame:
    with open(read_file, "r", encoding="utf8") as f:
        data = json.load(f)
    buffer = StringIO(data.get(key))
    return pd.read_csv(buffer, sep=",")

## Отдельно район

Изучается степенной закон внутри заданного района. Оригинальный подход их статьи.

In [ ]:
part_name = "Владимирская"
source_file = f"{part_name}.json"

### Обработка DataFrame

In [ ]:
df_reversed_edges = to_df(source_file, "reversed_edges_csv")
df_reversed_nodes = to_df(source_file, "reversed_nodes_csv")
reversed_g = gt.Graph()
reversed_g.add_edge_list(df_reversed_edges.loc[:, ["src_index", "dest_index"]].to_numpy())
df_reversed_nodes["degree"] = reversed_g.degree_property_map("out").get_array()
df_reversed_nodes = df_reversed_nodes[df_reversed_nodes["street_name"] != "Центральная улица"]
df_reversed_nodes = df_reversed_nodes[df_reversed_nodes["degree"] != 0]
degree = df_reversed_nodes["degree"]

#### ТОП-20 улиц с наибольшей связностью

In [ ]:
df_reversed_nodes.sort_values("degree", ascending=False).head(20)

### Выборочная функция распределения

#### Ручное ECDF

In [ ]:
def ecdf(x, log_scale=False):
    if log_scale:
        xs = np.log(np.sort(x))
        ys = np.log(np.arange(1, len(xs)+1)/float(len(xs)))
    else:
        xs = np.sort(x)
        ys = np.arange(1, len(xs)+1)/float(len(xs))
    return pd.DataFrame({"Degree": xs, "Proportion": ys[::-1]})

In [ ]:
df_degree = ecdf(degree.to_numpy())
df_log_degree = ecdf(degree.to_numpy(), log_scale=True)

### Линейная регрессия

Проверяем, возможно ли апроксимировать зависимость линейной функцией

In [ ]:
lr = LinearRegression().fit(df_log_degree["Degree"].to_numpy().reshape(-1, 1), df_log_degree["Proportion"])
c_l = lr.coef_[0]
i_l = lr.intercept_
x_start = df_log_degree["Degree"].iloc[0]
x_end = df_log_degree["Degree"].iloc[-1]
x_lr = [x_start, x_end]
y_lr = lr.predict(np.array([[x_start],[x_end]]))

### Полиномиальная регрессия (квадратичная)

Проверяем, возможно ли апроксимировать зависимость квадратичной функцией

In [ ]:
X = PolynomialFeatures(2).fit_transform(df_log_degree["Degree"].to_numpy().reshape(-1, 1))
pr = LinearRegression().fit(X, df_log_degree["Proportion"])
c_p = pr.coef_[1:]
i_p = pr.intercept_
x_pr = np.linspace(x_start, x_end, 20)
X_poly_features = PolynomialFeatures(2).fit_transform(x_pr.reshape(-1, 1))
y_pr = pr.predict(X_poly_features)

### Отрисовка

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 9))
fig.subplots_adjust(hspace=0.4)

x_linspace = np.linspace(1, df_degree["Degree"].max())
sns.lineplot(ax=ax[0], data=df_degree, x="Degree", y="Proportion", label="Фактическая", errorbar=None)
label = "Теоретическая $\\frac{1}{x^{" + str(-round(c_l, 5)) + "}}$"
sns.lineplot(ax=ax[0], x=x_linspace, y=x_linspace**c_l, label=label, errorbar=None)
ax[0].set_title("Стандартный")

sns.lineplot(ax=ax[1], data=df_log_degree, x="Degree", y="Proportion", label="Исходное", errorbar=None)
sns.lineplot(ax=ax[1], x=x_lr, y=y_lr, label=f"Линейная регрессия {c_l:.3f}*x + {i_l:.3f}", errorbar=None)
sns.lineplot(ax=ax[1], x=x_pr, y=y_pr, label=f"Квадратичная регрессия {c_p[1]:.3f}*x^2 + {c_p[0]:.3f}*x + {i_p:.3f}", errorbar=None)
ax[1].set_title("LogLog")

### Seaborn

Попытка отрисовать графики с использованием методов Seaborn. Лучше его не использовать

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 9))
fig.subplots_adjust(hspace=0.4)

sns.ecdfplot(ax=ax[0], data = df_reversed_nodes, x="degree", complementary=True)
ax[0].set_title("Стандартный")

sns.ecdfplot(ax=ax[1], data = df_reversed_nodes, x="degree", complementary=True, log_scale=True)
ax[1].set_title("LogLog")

## Район в контексте города

Изучается степенной закон для улиц заданного района, но их степени берутся на уровне всего города. Данный подход рассматривался на уровне идеи.

### Обработка DataFrame

In [ ]:
city_name = "СПб"
city_file = f"{city_name}.json"

In [ ]:
df_reversed_edges = to_df(city_file, "reversed_edges_csv")
df_reversed_nodes = to_df(city_file, "reversed_nodes_csv")
reversed_g = gt.Graph()
reversed_g.add_edge_list(df_reversed_edges.loc[:, ["src_index", "dest_index"]].to_numpy())
df_reversed_nodes["degree"] = reversed_g.degree_property_map("out").get_array()
df_reversed_nodes = df_reversed_nodes[df_reversed_nodes["street_name"] != "Центральная улица"]
df_reversed_nodes = df_reversed_nodes[df_reversed_nodes["degree"] != 0]

### Отбор улиц, принадлежащих району

In [ ]:
df_reversed_nodes_part = to_df(source_file, "reversed_nodes_csv")
df_reversed_nodes = df_reversed_nodes.loc[df_reversed_nodes["street_name"].isin(df_reversed_nodes_part["street_name"])]

degree = df_reversed_nodes["degree"]

#### ТОП-20 улиц с наибольшей связностью

In [ ]:
df_reversed_nodes.sort_values("degree", ascending=False).head(20)

### Выборочная функция распределения

In [ ]:
df_degree = ecdf(degree.to_numpy())
df_log_degree = ecdf(degree.to_numpy(), log_scale=True)

### Линейная регрессия

In [ ]:
lr = LinearRegression().fit(df_log_degree["Degree"].to_numpy().reshape(-1, 1), df_log_degree["Proportion"])
c_l = lr.coef_[0]
i_l = lr.intercept_
x_start = df_log_degree["Degree"].iloc[0]
x_end = df_log_degree["Degree"].iloc[-1]
x_lr = [x_start, x_end]
y_lr = lr.predict(np.array([[x_start],[x_end]]))

### Квадратичная регрессия

In [ ]:
X = PolynomialFeatures(2).fit_transform(df_log_degree["Degree"].to_numpy().reshape(-1, 1))
pr = LinearRegression().fit(X, df_log_degree["Proportion"])
c_p = pr.coef_[1:]
i_p = pr.intercept_
x_pr = np.linspace(x_start, x_end, 20)
X_poly_features = PolynomialFeatures(2).fit_transform(x_pr.reshape(-1, 1))
y_pr = pr.predict(X_poly_features)

### Отрисовка

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 9))
fig.subplots_adjust(hspace=0.4)

x_linspace = np.linspace(1, df_degree["Degree"].max())
sns.lineplot(ax=ax[0], data=df_degree, x="Degree", y="Proportion", label="Фактическая", errorbar=None)
label = "Теоретическая $\\frac{1}{x^{" + str(-round(c_l, 5)) + "}}$"
sns.lineplot(ax=ax[0], x=x_linspace, y=x_linspace**c_l, label=label, errorbar=None)
ax[0].set_title("Стандартный")

sns.lineplot(ax=ax[1], data=df_log_degree, x="Degree", y="Proportion", label="Исходное", errorbar=None)
sns.lineplot(ax=ax[1], x=x_lr, y=y_lr, label=f"Линейная регрессия {c_l:.3f}*x + {i_l:.3f}", errorbar=None)
sns.lineplot(ax=ax[1], x=x_pr, y=y_pr, label=f"Квадратичная регрессия {c_p[1]:.3f}*x^2 + {c_p[0]:.3f}*x + {i_p:.3f}", errorbar=None)
ax[1].set_title("LogLog")